# README? NoADME!


In [45]:
! pip install -qU gradio python-dotenv langchain-upstage
! pip install -qU requests

In [46]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [47]:
import warnings

warnings.filterwarnings("ignore")

# 2-shot prompt to get mermaid syntax

In [50]:
import gradio as gr

from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage()

smsg = """You are a helpful assistant. Please extract user and participants's name  from the following text. I am a user. Then, organize the action relationships between me and the participants.
---
    We are a team of 5 members called Krispy Team.
    Dr. Kim Yoon-gon gave me coffee. I said thank you.
    Student Jae-min also received coffee from Dr. Kim Yoon-gon. Dr. Son Seok-ho is explaining the team project topic now.
    Student Sung-hoon is preparing presentation materials for other team members.

    This is an answer template
    ```mermaid
    sequenceDiagram
        actor User
        participant Kim Yoon-gon
        participant Jae-min
        participant Son Seok-ho
        participant Sung-hoon
        participant Other team members

        User ->> Kim Yoon-gon : Received coffee and said thank you
        Kim Yoon-gon ->> Jae-min : Gave coffee
        Son Seok-ho ->> Team : Explained the project topic
        Sung-hoon ->> Other team members : Prepared presentation materials
    ```
    """

chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", smsg),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)


# Define functions to interact with GitHub

In [59]:
import requests
import json
import os
from datetime import datetime

# GitHub Personal Access Token 및 사용자/저장소 정보 설정
GITHUB_TOKEN = os.environ["GITHUB_TOKEN"]
REPO_OWNER = os.environ["REPO_OWNER"]
REPO_NAME = os.environ["REPO_NAME"]

# GitHub API URL 설정
GITHUB_API_URL = "https://api.github.com"

# 헤더 설정
headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

# 1. 브랜치 생성 함수
def create_branch(branch_name):
    # main 브랜치의 최신 커밋 SHA 가져오기
    response = requests.get(f"{GITHUB_API_URL}/repos/{REPO_OWNER}/{REPO_NAME}/git/ref/heads/main", headers=headers)
    sha = response.json()["object"]["sha"]

    # 새 브랜치 생성
    data = {
        "ref": f"refs/heads/{branch_name}",
        "sha": sha
    }
    response = requests.post(f"{GITHUB_API_URL}/repos/{REPO_OWNER}/{REPO_NAME}/git/refs", headers=headers, data=json.dumps(data))
    if response.status_code == 201:
        print(f"Branch '{branch_name}' created successfully.")
    else:
        print(f"Failed to create branch: {response.json()}")

# 2. 파일 커밋 함수
def commit_file(branch_name, file_path, file_content):
    # 파일을 Base64로 인코딩
    import base64
    encoded_content = base64.b64encode(file_content.encode()).decode()

    # 파일 커밋
    data = {
        "message": f"Add a generated diagram {file_path}",
        "content": encoded_content,
        "branch": branch_name
    }
    response = requests.put(f"{GITHUB_API_URL}/repos/{REPO_OWNER}/{REPO_NAME}/contents/{file_path}", headers=headers, data=json.dumps(data))
    if response.status_code == 201:
        print(f"File '{file_path}' committed successfully.")
    else:
        print(f"Failed to commit file: {response.json()}")

# 3. PR 생성 함수
def create_pull_request(branch_name):
    data = {
        "title": "Add a generated diagram",
        "head": branch_name,
        "base": "main",
        "body": "This PR adds a generated diagram written in Mermaid syntax."
    }
    response = requests.post(f"{GITHUB_API_URL}/repos/{REPO_OWNER}/{REPO_NAME}/pulls", headers=headers, data=json.dumps(data))
    if response.status_code == 201:
        print("Pull request created successfully.")
    else:
        print(f"Failed to create pull request: {response.json()}")


# Set knowledge base of Mermaid syntax

TBD

# Tool RAG

TBD

In [60]:
chain = chat_with_history_prompt | llm | StrOutputParser()

In [63]:
from langchain_upstage import UpstageGroundednessCheck

def gc(aimsg, hmsg):
    groundedness_check = UpstageGroundednessCheck()

    answer = aimsg
    gc_result = groundedness_check.invoke({"context": hmsg, "answer": answer})

    print(gc_result)
    if gc_result.lower().startswith("grounded"):
        return "[ Groundedness check passed ] "
    else:
        return "[ Groundedness check failed ]"

def chat(message, history):
    history_langchain_format = []
    # aimsg =""
    # hmsg =""
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    
    response = chain.invoke({"message": message, "history": history_langchain_format})
    
    ret = gc(response, message)
    
    ans = response + "\n" + ret
    print(ans)
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    f_path = f"test/generated-diagram-{timestamp}.md"
    branch = f"add-diagram-{timestamp}"
    create_branch(branch)
    commit_file(branch, f_path, ans)
    create_pull_request(branch)
    
    return ans

In [64]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "How to eat healthy?",
            "Best Places in Korea",
            "How to make a chatbot?",
        ],
        title="Solar Chatbot",
        description="Upstage Solar Chatbot",
    )
    chatbot.chatbot.height = 300

In [65]:
if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


grounded
```mermaid
sequenceDiagram
    participant User
    participant NoREADME Server
    participant GitHub
    participant LangChain
    participant Context Searcher
    participant Text Analyzer
    participant Entity Extractor
    participant Mermaid Diagram Generator
    participant Validator

    User->>NoREADME Server: 입력 GitHub md 파일 URL
    NoREADME Server->>LangChain: 파이프라인 시작
    LangChain->>URL Input: URL 입력
    URL Input->>LangChain: URL 반환
    LangChain->>File Download: 파일 다운로드
    File Download->>GitHub: md 파일 요청
    GitHub->>File Download: md 파일 제공
    File Download->>LangChain: 다운로드된 파일 콘텐츠 반환
    LangChain->>Context Searcher: 관련 컨텍스트 검색
    Context Searcher->>LangChain: 관련 컨텍스트 반환
    LangChain->>Text Analyzer: 파일 콘텐츠 및 컨텍스트 분석
    Text Analyzer->>LangChain: 추출된 컨텍스트 반환
    LangChain->>Entity Extractor: 엔티티 및 상호작용 추출
    Entity Extractor->>LangChain: 엔티티 및 상호작용 반환
    LangChain->>Mermaid Diagram Generator: Mermaid 다이어그램 생성
    Mermaid Diagram Generator->>LangChain: